Я взяла статьи с сайта https://cyberleninka.ru , ко всем были выделены ключевые слова

In [1]:
key_site_tangerine = 'листья / мандарин / числовые показатели'
key_site_football = 'футбол / обучение'
key_site_psycho = 'психология / история психологии / практика психологии / психотерапия'
key_site_dogs = 'ягдтерьер / натаска / интеллект / охота / кровяной след / подранки'

Добавим выделенные мной ключевые слова для каждой статьи:

In [28]:
key_tangerine = 'лист / мандарин / числовой показатель / лекарство '
key_football = 'футбол / обучение / воспитание / тренер / поведение'
key_psycho = 'психология / история психология / практика психология / психотерапия / ценность'
key_dogs = 'ягдтерьер / натаска / интеллект / охота / кровяной след / подранки / собака / норная собака / кровь / след'

Мне кажется, что для некоторых статей список ключевых слов были неполным, поэтому я буду опираться в том числе на добавленные мной слова?словосочетания

In [3]:
import re
import pandas as pd
from statistics import mean 
from textdistance import levenshtein
from nltk.tokenize import RegexpTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('russian')
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/darazharikova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
texts_dict = {}
texts_keys ={}

with open('tangerine.txt', 'r', encoding='utf-8') as text:
            text = text.read()
            texts_dict['tangerine'] = text
#             texts_keys [key_tangerine] = text
            texts_keys ['tangerine'] = key_tangerine
with open('football.txt', 'r', encoding='utf-8') as text:
            text = text.read()
            texts_dict['football'] = text
#             texts_keys [key_football] = text
            texts_keys ['football'] = key_football
with open('psycho.txt', 'r', encoding='utf-8') as text:
            text = text.read()
            texts_dict['psycho'] = text
            texts_keys ['psycho'] = key_psycho
#             texts_keys [key_psycho] = text
with open('dogs.txt', 'r', encoding='utf-8') as text:
            text = text.read()
            texts_dict['dogs'] = text
#             texts_keys [key_psycho] = text
            texts_keys ['dogs'] = key_dogs


Любимый препроцессинг

In [5]:
tokenizer = RegexpTokenizer(r'\w+')
def preproc(text, stopwords, maximal=False):
    stopwords = set(stopwords)
    text = re.sub(r'\n', r' ', text)
    if maximal:
        text = re.sub(r'["%0-9A-Za-z]', r'', text)
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    newtext = ''
    for w in text:
        if w not in stopwords: 
            lemma = morph.parse(w)[0].normal_form + ' '
            newtext += lemma
    return newtext

preproc_texts_dict = {}
for t in texts_dict.keys():
    lemmatized = preproc(texts_dict[t], stop, maximal=False)
    preproc_texts_dict[t] = lemmatized

In [6]:
# preproc_texts_dict

In [7]:
def keys_simple_search(keywords, text_dict):
    for n in keywords.keys():
        print(n)
        text = text_dict[n]
        keys = keywords[n]
        keys = keys.split (' / ')
        for kw in keys:
            kw_c = text.count(kw)
            print(f'{kw} - {kw_c}')
        print('\n')

In [8]:
keys_simple_search(texts_keys, preproc_texts_dict)

tangerine
листья - 0
мандарин - 15
числовые показатели - 0
лекарство  - 0


football
футбол - 33
обучение - 0
воспитание - 3
тренер - 6
поведение - 3


psycho
психология - 10
история психологии - 0
практика психологии - 0
психотерапия - 11
ценность - 2


dogs
ягдтерьер - 15
натаска - 2
интеллект - 0
охота - 4
кровяной след - 22
подранки - 0
собака - 43
норная собака - 0
кровь - 12




Видим, что некоторые ключевые слова и словосочетания встречаются не так часто => интересно посмотреть, выделят ли их наши методы

In [9]:
# !pip install python-RAKE

# Rake

In [10]:
import RAKE
rake = RAKE.Rake(stop)

In [11]:
def preproc_rake(text, maximal=False):
    text = re.sub(r'\n', r' ', text)
    if maximal:
        text = re.sub(r'["%0-9A-Za-z]', r'', text)
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    newtext = ''
    for w in text:
        lemma = morph.parse(w)[0].normal_form + ' '
        newtext += lemma
    return newtext

In [12]:
rake_preproc_texts_dict = {}
for t in texts_dict.keys():
    lemmatized = preproc_rake(texts_dict[t], maximal=False)
    rake_preproc_texts_dict[t] = lemmatized

In [13]:
def metrics(pred, real):
    tp = 0
    fn = 0
    fp = 0
    #tn
    for p in pred:
        if p in real:
            tp += 1 #3
        if p not in real:
            fp += 1 #2
    for i in real:
        if i not in pred:
            fn += 1 #5
    accuracy = tp / (tp + fn + fp)
    precision = tp / (tp + fp) #3/3+2
    recall = tp / (tp + fn)
    try:
        f1 = (2 * precision * recall) / (precision + recall)
    except ZeroDivisionError:
        f1 = 0.0
    
    print('accuracy: ' + str(accuracy) + '\n' + 'precision: ' + str(precision) + '\n' + 'f1: ' + str(f1))

In [14]:
keys_rake_pred_dict = {}

for text in texts_dict.keys():
    print(text)
    rake_kw_list = rake.run(preproc_texts_dict[text], maxWords=3, minFrequency=1)
    kw_list = []
    for kw in rake_kw_list:
        print(kw)
        kw_list.append(kw[0])
    keys_rake_pred_dict[text] = kw_list
    print('\n')

tangerine
('р органический кислота', 9.0)
('витамин', 1.0)


football
('спортивный воспитание футбол', 9.0)


psycho
('тенденция рисковать привести', 9.0)
('являться носитель эталон', 9.0)
('нашея профессией', 4.0)
('активно навязывать', 4.0)
('психический', 1.0)
('нож', 1.0)
('руководить', 1.0)
('первое', 1.0)


dogs
('испытание прокладывание', 4.0)
('проявление анонс', 4.0)
('часовой давность', 4.0)
('дильник', 1.0)
('кровь', 1.0)
('ягдтерьер', 1.0)
('собака', 1.0)
('область', 1.0)




In [30]:
def do_metrics(pred_dict):
    global texts_keys
    for text in texts_keys.keys():
        keys_real = texts_keys[text]
        keys_real = keys_real.split (' / ')
        print (keys_real)
        keys_pred = pred_dict[text]
        print (keys_pred)
        do_metrics = metrics (keys_real,keys_pred)
    return
do_metrics(keys_rake_pred_dict)

['лист', 'мандарин', 'числовой показатель', 'лекарство ']
['р органический кислота', 'витамин']
accuracy: 0.0
precision: 0.0
f1: 0.0
['футбол', 'обучение', 'воспитание', 'тренер', 'поведение']
['спортивный воспитание футбол']
accuracy: 0.0
precision: 0.0
f1: 0.0
['психология', 'история психология', 'практика психология', 'психотерапия', 'ценность']
['тенденция рисковать привести', 'являться носитель эталон', 'нашея профессией', 'активно навязывать', 'психический', 'нож', 'руководить', 'первое']
accuracy: 0.0
precision: 0.0
f1: 0.0
['ягдтерьер', 'натаска', 'интеллект', 'охота', 'кровяной след', 'подранки', 'собака', 'норная собака', 'кровь', 'след']
['испытание прокладывание', 'проявление анонс', 'часовой давность', 'дильник', 'кровь', 'ягдтерьер', 'собака', 'область']
accuracy: 0.2
precision: 0.3
f1: 0.33333333333333326


Видим, что rake выделяет часто словосочетания. Для статьи football это даже неплохо получилось, но для статьи psycho не очень - возможно потому что статья сама по себе очень разнообразна. (я попыталась подправить ситацию со словосоетаниями, чтобы лучше работали метрики, но юпитер ломается на этом месте каждый раз. Пишу чтобы вы не подумали, что я не вижу проблему с подсчетом точности и тд)

# Textrank

In [16]:
from gensim.summarization import keywords as kw

In [31]:
keys_textrank_pred_dict = {}

for text in texts_dict.keys():
    print(text)
    text_rank = kw(preproc_texts_dict[text], pos_filter=[], scores=True)
    kw_list = []
    for k_w in text_rank[:5]:
        print(f'{k_w[0]}\t{k_w[1]}')
        kw_list.append(k_w[0])
    keys_textrank_pred_dict[text] = kw_list
    print('\n')

tangerine
плод	0.20636922780727826
которыи	0.18127954548032452
род цитрус мандарин	0.1687535930777063
лист	0.1379554152191496
также	0.10829621999634018


football
спортивныи	0.2356606420445973
спорт	0.16183141593875067
юныи	0.1605121572012145
профессиональныи футболист это личность	0.15933995762440142
свои	0.15897264592256943


psycho
которыи	0.2826962774371547
это	0.22330098740377044
психология	0.20195192739453083
психотерапия	0.17584963333746262
направление подход	0.14319714341420411


dogs
след	0.33100144433714807
охотничии собака	0.2842191846832642
это	0.1832984036145123
ягдтерьер	0.15838015601436595
кровь	0.14551358782885038




In [32]:
do_metrics(keys_textrank_pred_dict)

['лист', 'мандарин', 'числовой показатель', 'лекарство ']
['плод', 'которыи', 'род цитрус мандарин', 'лист', 'также']
accuracy: 0.125
precision: 0.25
f1: 0.22222222222222224
['футбол', 'обучение', 'воспитание', 'тренер', 'поведение']
['спортивныи', 'спорт', 'юныи', 'профессиональныи футболист это личность', 'свои']
accuracy: 0.0
precision: 0.0
f1: 0.0
['психология', 'история психология', 'практика психология', 'психотерапия', 'ценность']
['которыи', 'это', 'психология', 'психотерапия', 'направление подход']
accuracy: 0.25
precision: 0.4
f1: 0.4000000000000001
['ягдтерьер', 'натаска', 'интеллект', 'охота', 'кровяной след', 'подранки', 'собака', 'норная собака', 'кровь', 'след']
['след', 'охотничии собака', 'это', 'ягдтерьер', 'кровь']
accuracy: 0.25
precision: 0.3
f1: 0.4


Тоже неплохо! Но здесь, кажется, попадает больше шелухи (как например в dogs "это" выше чем "собака").

# tf-Idf

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [34]:
corpus = list(preproc_texts_dict.values())

cv=CountVectorizer(max_features=10000) #stop_words=stopwords,
word_count_vector=cv.fit_transform(corpus)

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tf_idf_vector=tfidf_transformer.fit_transform(word_count_vector)

df = pd.DataFrame(tf_idf_vector.toarray(), columns=cv.get_feature_names(), index=texts_dict.keys())

stop = set(stop)
words = set(df.columns)

stop_words = stop.intersection(words)
for w in stop_words:
    df = df.drop([w], axis=1)

df = df.T

In [44]:
col_names = list(df.columns)

keys_pred_tfidf_dict = {}

for col in col_names:
#     len_k = len(texts_keys[col].split (' / '))
    kws = df.nlargest(7, col)
    kws = kws[col]
    print(col)
    print(kws)
    keys_pred_tfidf_dict[col] = list(kws.index)
    print('\n')

tangerine
мандарин    0.514641
плод        0.257320
лист        0.183800
кожура      0.147040
который     0.134280
citrus      0.110280
витамин     0.110280
Name: tangerine, dtype: float64


football
футболист     0.398241
футбольный    0.284458
юный          0.284458
футбол        0.256012
спортивный    0.246697
спорт         0.170675
спортсмен     0.170675
Name: football, dtype: float64


psycho
психотерапия    0.281638
психология      0.256034
направление     0.201860
который         0.187053
это             0.179766
заложить        0.153621
подход          0.153621
Name: psycho, dtype: float64


dogs
собака       0.492642
след         0.469728
кровяной     0.263506
испытание    0.229136
охотничий    0.171852
ягдтерьер    0.171852
работа       0.138942
Name: dogs, dtype: float64




In [45]:
do_metrics(keys_pred_tfidf_dict)

['лист', 'мандарин', 'числовой показатель', 'лекарство ']
['мандарин', 'плод', 'лист', 'кожура', 'который', 'citrus', 'витамин']
accuracy: 0.2222222222222222
precision: 0.5
f1: 0.36363636363636365
['футбол', 'обучение', 'воспитание', 'тренер', 'поведение']
['футболист', 'футбольный', 'юный', 'футбол', 'спортивный', 'спорт', 'спортсмен']
accuracy: 0.09090909090909091
precision: 0.2
f1: 0.16666666666666666
['психология', 'история психология', 'практика психология', 'психотерапия', 'ценность']
['психотерапия', 'психология', 'направление', 'который', 'это', 'заложить', 'подход']
accuracy: 0.2
precision: 0.4
f1: 0.3333333333333333
['ягдтерьер', 'натаска', 'интеллект', 'охота', 'кровяной след', 'подранки', 'собака', 'норная собака', 'кровь', 'след']
['собака', 'след', 'кровяной', 'испытание', 'охотничий', 'ягдтерьер', 'работа']
accuracy: 0.21428571428571427
precision: 0.3
f1: 0.3529411764705882


Видим что здесь вообще всё еще лучше! Но статья psycho опять страдает

# Выводы

1) Несмотря на плачевные результаты метрик, методы справились неплохо. Лушче всего tf-idf, затем textrank и rake на 3м месте.
2) Rake больше остальных склонен к выделению ключевых последовательностей (больше 1 слова), возможно в этом его проблема. Хотя на бОльших текстах это может сработать лучше. А tf-idf выделяет только по 1 ключевому слову и для наших простеньких статей этого достаточно
3) В статье psycho использовалось много синонимов и тд - вот у нее везде и самых плохой результат, а в dogs использовались незаменимые слова (порода собаки "ягдтерьер", термин "кровяной след" и тд) и эту статью все методы более или менее хорошо обрабатывали.